In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, ConcatDataset, Subset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg  
from matplotlib.image import imread 
import seaborn as sns

from time import time
from tqdm.notebook import tqdm
from copy import deepcopy
    

In [ ]:
seed = 1337
batch_size = 200
resize = (200, 200)
rows_num = 10
test_split = 0.2
num_epoch = 5
learning_rate = 0.001
gamma = 0.1
patience = 3
learning_rate_unfreeze = 0.0001
steps_to_checkpoint = 1

In [ ]:
data_dir = "../input/pokemonclassification/PokemonData"
info_dir = "./"
file_first_name = "NN_practice_2"

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Visualizing dataset for augmentation

figure_size = plt.rcParams["figure.figsize"]
figure_size[0] = 7
figure_size[1] = 7
plt.rcParams["figure.figsize"] = figure_size

In [ ]:
img = mpimg.imread("../input/pokemonclassification/PokemonData/Abra/0282b2f3a22745f1a436054ea15a0ae5.jpg")
print(f"{img.shape}")
plt.imshow(img);


In [ ]:
def find_mean_std(path):
    dataset = ImageFolder(path, 
                          transform=transforms.Compose([
                              transforms.ToTensor(), 
                              transforms.Resize(resize)])
                          )
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False
    )
    mean = 0.0
    std = 0.0
    nb_samples = 0
    for data, labels in tqdm(loader):
        data.to(device)
        batch_samples = len(data)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples
    return mean, std


In [ ]:
mean = torch.tensor([0.6052, 0.5874, 0.5538])
std = torch.tensor([0.2507, 0.2409, 0.2486])

In [ ]:
def show_batch(dataset):
    fig, ax = plt.subplots(figsize=(50, 50))
    for images, labels in dataset:
        ax.set_xticks([]), ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=rows_num).permute(1, 2, 0))
        break;

In [ ]:
transformer = transforms.Compose([
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])
transformer_1 = transforms.Compose([
    transformer,
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomErasing(scale=(0.01, 0.5)),
    transforms.ColorJitter(brightness=(0.4, 1), contrast=(0.5, 0.9),
                           saturation=(0.5, 0.9), hue=(-0.1, 0.1)),
    transforms.RandomRotation(degrees=(0, 180)),
])
transformer_2 = transforms.Compose([
    transformer,
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(size=resize, scale=(0.25, 0.95)),
    transforms.RandomErasing(scale=(0.01, 0.25)),
    transforms.RandomAffine(degrees=(0, 75), translate=(0.3,0.3), scale=(0.7,0.7)),
])


In [ ]:
original_set = ImageFolder(data_dir, transform=transformer)
original_dataset = DataLoader(original_set, batch_size=batch_size,
                              pin_memory=True, shuffle=True)
show_batch(original_dataset)
classes = original_set.classes
del original_set
del original_dataset

In [ ]:
set_1 = ImageFolder(data_dir, transform=transformer_1)
dataset_1 = DataLoader(set_1, batch_size=batch_size,
                              pin_memory=True, shuffle=True)
show_batch(dataset_1)
del dataset_1

In [ ]:
set_2 = ImageFolder(data_dir, transform=transformer_2)
dataset_2 = DataLoader(set_2, batch_size=batch_size,
                              pin_memory=True, shuffle=True)
show_batch(dataset_2)
del dataset_2

In [ ]:
def gen_random_indices(dataset_len, persentage):
    indices = list(range(dataset_len))
    split = int(np.floor(persentage * dataset_len))
    np.random.seed(seed)
    np.random.shuffle(indices)
    train_idx, test_idx = indices[split:], indices[:split]
    return train_idx, test_idx

In [ ]:
def train_val_test_split(data_dir, valid_persentage):
    datafolder = ImageFolder(data_dir, transformer)
    train_val_indexes, test_indexes = gen_random_indices(len(datafolder), valid_persentage)
    test = Subset(datafolder, indices=test_indexes)
    train_val = Subset(datafolder, indices=train_val_indexes)
    dataset = ConcatDataset([
        train_val,
        set_1,
        set_2
    ])
    train_indexes, val_indexes = gen_random_indices(len(dataset), valid_persentage)
    return {
        "train": DataLoader(dataset, batch_size=batch_size, num_workers=4,
                            pin_memory=True, sampler=SubsetRandomSampler(train_indexes)),
        "validation": DataLoader(dataset, batch_size=batch_size, num_workers=4,
                            pin_memory=True, sampler=SubsetRandomSampler(val_indexes)),
        "test": DataLoader(test, batch_size=batch_size, num_workers=4,
                            pin_memory=True, shuffle=True),
    }

In [ ]:
original = ImageFolder(path, transform=transformer['original'])

#all_set = train_val + test
train_val, test = train_test_split(original, test_size=test_split, shuffle=True, random_state=seed)

#train_val = train + val + dataset1 + dataset2 + dataset3
train_val = ConcatDataset([train_val, 
                           ImageFolder(path, transform=transformer['dataset1']),
                           ImageFolder(path, transform=transformer['dataset2']),
                           ImageFolder(path, transform=transformer['dataset3'])]) 

train, val = train_test_split(train_val, test_size=test_split, shuffle=True, random_state=seed)

dataloaders = {
    'train': DataLoader(train, batch_size=bs, num_workers=4, pin_memory=True),
    'val': DataLoader(val, batch_size=bs, num_workers=4, pin_memory=True),
    'test': DataLoader(test, batch_size=bs, num_workers=4, pin_memory=True)
}

In [ ]:
dataset_sizes

In [ ]:
resnet101 = models.wide_resnet101_2(pretrained=True, progress=True)
for params in resnet101.parameters():
    params.requires_grad = False 
resnet101.fc = nn.Linear(in_features=resnet101.fc.in_features, out_features=len(classes), bias=True);

In [ ]:
densenet161 = torchvision.models.densenet161(pretrained=True, progress=True)
for params in densenet161.parameters():
    params.requires_grad=False
densenet161.classifier = nn.Linear(in_features=densenet161.classifier.in_features, out_features=len(classes), bias=True)

In [ ]:
large_mobilenet = models.mobilenet_v2(pretrained=True, progress=True)
for param in large_mobilenet.parameters():
    param.requires_grad=False
large_mobilenet.classifier[1] = nn.Linear(in_features=large_mobilenet.classifier[1].in_features, out_features=len(classes), bias=True)

In [ ]:
googlenet = models.googlenet(pretrained=True, progress=True)
for param in googlenet.parameters():
    param.requires_grad = False
googlenet.fc = nn.Linear(in_features=googlenet.fc.in_features, out_features=len(classes), bias=True)

In [ ]:
vgg19 = models.vgg19_bn(pretrained=True, progress=True)
for param in vgg19.parameters():
    param.required_grad = False
vgg19.classifier[6] = nn.Linear(in_features=vgg19.classifier[6].in_features, out_features=len(classes), bias=True)

In [ ]:
mnas = torchvision.models.mnasnet1_0(pretrained=True, progress=True)
for param in mnas.parameters():
    param.required_grad = False
mnas.classifier[1] = nn.Linear(in_features=mnas.classifier[1].in_features, out_features=len(classes), bias=True)

In [ ]:
len(vgg19.classifier), len(large_mobilenet.classifier), len(mnas.classifier)

In [ ]:
checkpoint = {
    "epoch" : 0,
    "name" : "",
    "optim_params" : {},
    "criterion_params" : {},
    "model": {},
    "accuracy": 0.0,
    "scheduler": {}
}

In [ ]:
def copy_state(epoch, name, optim, criterion, model, accuracy, scheduler):
    state = deepcopy(checkpoint)
    state["epoch"] = epoch
    state["name"] = name
    state["optim"] = deepcopy(optim.state_dict())
    state["criterion"] = deepcopy(criterion.state_dict())
    state["model"]  = deepcopy(model.state_dict())
    state["scheduler"] = deepcopy(scheduler.state_dict())
    state["accuracy"] = accuracy
    return state

In [ ]:
def load_state(checkpoint, map_location=device):
    optimizer = torch.load(checkpoint["optim"], map_location=map_location)
    criterion = torch.load(checkpoint["criterion"], map_location=map_location)
    model = torch.load(checkpoint["model"], map_location=map_location)
    return optimizer, criterion, model

In [ ]:
def save_to_drive(file_name, data_dir, statement):
    path = f"{data_dir}/{file_first_name}{file_name}.pth"
    torch.save(statement, path)

In [ ]:
def load_from_drive(file_name, data_dir, map_location=device):
    path = f"{data_dir}/{file_first_name}{file_name}.pth"
    statement = torch.load(path, map_location=map_location) if open(path) else None
    return statement

In [ ]:
def train(model, model_name, num_epochs, seed, optimizer):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=patience, verbose=True)
    state_history = {
        "train" : {
            "loss": [],
            "acc" : []
            },
        "val" : {
            "loss": [],
            "acc" : []
            }
    }
    statement = deepcopy(checkpoint)
    start = time()
    for epoch in tqdm(range(num_epochs)):
        for phase in tqdm(["train", "validation"]):
            if phase == "train":
                model.train()
            else:
                model.eval()
            current_loss = 0.0
            current_correct = 0
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                with torch.set_grad_enabled(phase=="train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == "train":
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                current_loss += loss.item() * inputs.size(0)
                current_correct += torch.sum(preds==labels.data)
            index = 0 if phase == "train" else 1
            epoch_loss = current_loss / (dataset_sizes[index] * batch_size)
            epoch_accuracy = current_correct.double() / (dataset_sizes[index] * batch_size)
            if phase == "train":
                scheduler.step(epoch_accuracy * 100)
            print(f"{phase} loss: {epoch_loss:.4f} accuracy: {epoch_accuracy:.4f}")
            if phase == "validation":
                state_history["val"]["loss"] = epoch_loss
                state_history["val"]["acc"] = epoch_accuracy
                if epoch_accuracy > statement["accuracy"]:
                    statement = copy_state(epoch, model_name, optimizer,
                                           criterion, model, epoch_accuracy, scheduler)
            else:
                state_history["train"]["loss"] = epoch_loss
                state_history["train"]["acc"] = epoch_accuracy
            # saving info
            if (epoch + 1) % steps_to_checkpoint == 0:
                save_to_drive(model_name, info_dir, statement)
            torch.cuda.empty_cache()
    time_passed = time() - start
    print(f"Training complete in {time_passed//60}m:{int(time_passed%60)}s")
    print(f"Best accuracy is: {statement['accuracy']}")
    model.load_state_dict(statement["model"])
    return model, state_history

In [ ]:
list_of_models = [large_mobilenet, resnet101, densenet161, googlenet, vgg19, mnas]
names_of_model = ["mobilenet", "resnet", "densenet", "googlenet", "vgg", "mnas"]
models_fc = ["resnet", "googlenet"]
statements_freeze = []
statements_unfreeze = []
for index in tqdm(range(len(names_of_model))):
    optimizer= 0
    if names_of_model[index] in models_fc:
        optimizer = torch.optim.Adam(list_of_models[index].fc.parameters(), lr=learning_rate)
    else:
        optimizer = torch.optim.Adam(list_of_models[index].classifier.parameters(), lr=learning_rate)
    _, statements = train(list_of_models[index], names_of_model[index], num_epoch, seed, optimizer)
    statements_freeze.append(statements)
    ###UNFREZE TECHNIQUE 
    for param in list_of_models[index].parameters():
        param.requires_grad = True
    optimizer = torch.optim.Adam(list_of_models[index].parameters(), lr=learning_rate_unfreeze)
    _, statement = train(list_of_models[index], names_of_model[index], num_epoch, seed, optimizer)
    statements_unfreeze.append(statement)

In [ ]:
class Ensemble(nn.Module):
    def __init__(self, device):
        super(Ensemble, self).__init__()
        self.models = nn.ModuleList(list_of_models).to(device)
    def forward(self, x):
        output = torch.zeros([x.size(0), len(classes)]).to(device)
        for model in self.models:
            output += model(x)
        return output

In [ ]:
model = Ensemble(device)

In [ ]:
@torch.no_grad()
def test_prediction(model, test_loader):
    model.eval()
    loss = []
    accuracy = [] 
    predictions = []
    labels_list = []
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        output.to("cpu")
        _, preds = torch.max(output, dim=1)
        acc = torch.tensor(torch.sum(preds==labels).item() / len(preds))
        loss.append(nn.functional.cross_entropy(output, labels))
        accuracy.append(acc)
        predictions.append(preds.tolist())
        labels_list.append(labels.tolist())
    loss = torch.stack(loss).mean()
    accuracy = torch.stack(accuracy).mean()
    return loss, accuracy, predictions, labels_list

In [ ]:
loss, accuracy, predicts, labels = test_prediction(model, dataloaders["test"])
f"Loss is: {loss} and accuracy is: {accuracy*100} %"